In [1]:
import pymc3 as pm
import numpy as np
import theano

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n_=theano.shared(np.asarray([10, 15]))
with pm.Model() as m:
    p = pm.Beta('p', 1., 1.)
    ni = pm.Bernoulli('ni', .5)
    k = pm.Binomial('k', p=p, n=n_[ni], observed=4)
    trace = pm.sample()

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [p_logodds__]
>BinaryGibbsMetropolis: [ni]
100%|██████████| 1000/1000 [00:00<00:00, 1129.41it/s]
INFO (theano.gof.compilelock): Waiting for existing lock by process '20355' (I am process '20356')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/laoj/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-3.5.2-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '20355' (I am process '20357')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/laoj/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-3.5.2-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '20356' (I am process '20357')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/laoj/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-3.5.2-64/lock_dir
The acceptance probab

In [3]:
m.free_RVs

[p_logodds__, ni]

In [4]:
with m:
    step1 = pm.Metropolis([m.free_RVs[0]])
    step2 = pm.BinaryGibbsMetropolis([ni])

In [6]:
with m:
    comp_step1 = pm.CompoundStep([step1, step2])
comp_step1.methods

In [7]:
with m:
    comp_step2 = pm.CompoundStep([step2, step1])
comp_step2.methods

In [180]:
point = m.test_point
point

{'ni': array(0), 'p_logodds__': array(0.)}

In [186]:
point, state = step1.step(point=point)
point, state

({'ni': array(0), 'p_logodds__': array(0.69089502)},
 [{'accept': 0.8568568614073855, 'tune': True}])

In [187]:
point = step2.step(point=point)
point

{'ni': array(0), 'p_logodds__': array(0.69089502)}

In [16]:
cpt = np.array([[1., 0., 0.], [0, 1, 0], [0, 0, 1]])
cpt = theano.shared(cpt)
q = cpt[a]

with pm.Model() as model:
    p = np.array([0.5, 0.2, 0.3])
    a = pm.Categorical('a', p=p)

    b = pm.Categorical('b', p=q)
    step = pm.Metropolis()
    tr = pm.sample(500, step=step)

a_samples = tr['a']

print('0: {}'.format(len([s for s in a_samples if s[0] == 0])))
print('1: {}'.format(len([s for s in a_samples if s[0] == 1])))
print('2: {}'.format(len([s for s in a_samples if s[0] == 2])))

ValueError: too many values to unpack (expected 1)

In [8]:
%debug

> /Users/jlao/Documents/Github/pymc3/pymc3/theanof.py(272)__call__()
    270         input : TensorVariable
    271         """
--> 272         oldinput, = inputvars(self.tensor)
    273         return theano.clone(self.tensor, {oldinput: input}, strict=False)
    274 

ipdb> up
> /Users/jlao/Documents/Github/pymc3/pymc3/step_methods/metropolis.py(629)delta_logp()
    627     inarray1 = tensor_type('inarray1')
    628 
--> 629     logp1 = pm.CallableTensor(logp0)(inarray1)
    630 
    631     f = theano.function([inarray1, inarray0], logp1 - logp0)

ipdb> inarray1
inarray1
ipdb> logp0
__logp
ipdb> up
> /Users/jlao/Documents/Github/pymc3/pymc3/step_methods/metropolis.py(136)__init__()
    134 
    135         shared = pm.make_shared_replacements(vars, model)
--> 136         self.delta_logp = delta_logp(model.logpt, vars, shared)
    137         super(Metropolis, self).__init__(vars, shared)
    138 

ipdb> vars
[b]
ipdb> shared
{a: a_shared}
ipdb> q
